In [1]:
#A tisztított adatok feltöltése PostgreSQL adatbázisba az sqlalchemy segítségével.
from sqlalchemy import create_engine, text
import pandas as pd

import os
from dotenv import find_dotenv, load_dotenv

In [2]:
dotenv_path = find_dotenv() 

load_dotenv(dotenv_path)

#API_KEY = os.getenv("API_KEY")
USER = os.getenv("USER")
PASSWORD = os.getenv("PASSWORD")
LOCALHOST = os.getenv("LOCALHOST")
#PORT = os.getenv("PORT")
DATABASE = os.getenv("DATABASE")
DRIVER = os.getenv("DRIVER")

In [ ]:
engine = create_engine(f"mssql+pyodbc://{USER}:{PASSWORD}@{LOCALHOST}/{DATABASE}?driver={DRIVER}" ) # .env

print("Kapcsolat létrehozva az adatbázissal!")

# Kapcsolat tesztelése
with engine.connect() as connection:
    result = connection.execute(text("SELECT @@VERSION"))
    for row in result:
        print(row)

Kapcsolat létrehozva az adatbázissal!
('Microsoft SQL Server 2019 (RTM) - 15.0.2000.5 (X64) \n\tSep 24 2019 13:48:23 \n\tCopyright (C) 2019 Microsoft Corporation\n\tEnterprise Evaluation Edition (64-bit) on Windows 10 Pro 10.0 <X64> (Build 19045: ) (Hypervisor)\n',)


In [4]:
# Adatok feltöltése SSMS-be
df = pd.read_csv(r'C:\Users\Évi\OneDrive\Asztali gép\Portfoliio project\Data Science Folder Template (jelszólevédés)\project-name\data\raw\Clean_Dataset.csv', delimiter = ',')
#df = pd.read_csv(r'C:\Users\Évi\source\repos\Data Science Folder Template\project-name\data\continuous_factory_process.csv', encoding='utf-8')

tablename = 'flight_clean_table'

df.to_sql(tablename, engine, if_exists='replace', index=True)
print("Adatok feltöltve az adatbázisba!")

Adatok feltöltve az adatbázisba!


In [ ]:
tablename = 'flight_clean_table' 
sql_df = pd.read_sql(tablename, engine )
sql_df.head()

,index,Unnamed: 0,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,0,0,SpiceJet,SG-8709,Delhi,Evening,zero,Night,Mumbai,Economy,2.17,1,5953
1,1,1,SpiceJet,SG-8157,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2.33,1,5953
2,2,2,AirAsia,I5-764,Delhi,Early_Morning,zero,Early_Morning,Mumbai,Economy,2.17,1,5956
3,3,3,Vistara,UK-995,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2.25,1,5955
4,4,4,Vistara,UK-963,Delhi,Morning,zero,Morning,Mumbai,Economy,2.33,1,5955


In [ ]:
# Feltöltött táblából lekérdezés
df_query = pd.read_sql_query("SELECT * FROM flight_clean_table WHERE airline = 'Vistara' " , engine)
print(df_query.head())

   index  Unnamed: 0  airline  flight source_city departure_time stops  \
0      3           3  Vistara  UK-995       Delhi        Morning  zero   
1      4           4  Vistara  UK-963       Delhi        Morning  zero   
2      5           5  Vistara  UK-945       Delhi        Morning  zero   
3      6           6  Vistara  UK-927       Delhi        Morning  zero   
4      7           7  Vistara  UK-951       Delhi      Afternoon  zero   

  arrival_time destination_city    class  duration  days_left  price  
0    Afternoon           Mumbai  Economy      2.25          1   5955  
1      Morning           Mumbai  Economy      2.33          1   5955  
2    Afternoon           Mumbai  Economy      2.33          1   5955  
3      Morning           Mumbai  Economy      2.08          1   6060  
4      Evening           Mumbai  Economy      2.17          1   6060  


In [5]:
# SSMS adatbázisból tábladatok lekérése (SQL-lekérdezések futtatása pandasban)

tablenames = ['Szallashely', 'Szoba', 'Vendeg', 'Foglalas' ]
for tablename in tablenames:
    sql_df = pd.read_sql(f"SELECT * FROM {tablename}", engine)  # Itt egyesével kérdezzük le a táblákat
    print(sql_df.head())

   SZALLAS_ID       SZALLAS_NEV              HELY  CSILLAGOK_SZAMA      TIPUS  \
0           1          Sába-Ház       Balaton-dél                0  vendégház   
1           2       Családi Ház       Balaton-dél                0  vendégház   
2           3  Fortuna Apartman  Hajdúbihar megye                0   Apartman   
3           4    Fortuna panzió          Budapest                3     panzió   
4           5    Fortuna Panzió       Békés megye                3     panzió   

  ROGZITETTE    ROGZ_IDO                                          CIM  
0       Béla  2016-02-28    8630 Balatonboglár, Radnóti Miklós utca 8  
1       Béla  2016-03-02    8630 Balatonboglár, József Attila utca 25  
2       Sára  2016-03-06     4200 Hajdúszoboszló, Wesselényi utca 56.  
3       Béla  2016-03-07           1019 Budapest XIV. Cinkotai út 86.  
4       Sára  2016-03-08  5900 Orosháza, Gyopárosfürdő Tópart utca 3.  
   SZOBA_ID SZOBA_SZAMA  FEROHELY  POTAGY KLIMAS  SZALLAS_FK
0         1         

In [6]:
# 01.Készítsünk listát arról, hogy melyik vendég hányszor foglalt szállást összesen. 
# a. A lista tartalmazza a végösszeget is. 
# b. A listát rendezzük a rendelések száma szerint növekvő sorrendbe!

df_query = pd.read_sql_query('''select iif(Grouping_ID(vendeg.nev)=1,'Végösszeg', cast(vendeg.nev as nvarchar(20))) as 'Vendég', 
count(Foglalas.FOGLALAS_PK) as 'Foglalások száma' 
from Vendeg join Foglalas on Vendeg.USERNEV = Foglalas.UGYFEL_FK
group by rollup(vendeg.nev) order by 'Foglalások száma';''' , engine)
print(df_query)

             Vendég  Foglalások száma
0       Berta Dávid                 1
1    Bodor Szabolcs                 1
2      Dániel Tibor                 1
3     Farkas László                 1
4       Nagy András                 1
..              ...               ...
191    Barna Zoltán                12
192     Farkas Sára                12
193  Komjáti András                13
194    Kelemen Áron                13
195       Végösszeg              1027

[196 rows x 2 columns]


In [7]:
# 02 Készítsünk lekérdezést, amely a foglalást leadott vendégek adatait, a foglalások hosszát, továbbá az adott ügyfél legrövidebb, 
# illetve leghosszabb foglalásának hosszát jeleníti meg!

df_query = pd.read_sql_query('''select vendeg.nev, Foglalas.FOGLALAS_PK ,DATEDIFF(DAY, foglalas.mettol, foglalas.meddig) AS 'Foglalás hossza',
	min(datediff(day, foglalas.mettol, foglalas.meddig)) over(partition by foglalas.ugyfel_fk) as 'Ügyfél legrövidebb foglalása',
	max(datediff(day, foglalas.mettol, foglalas.meddig)) over(partition by foglalas.ugyfel_fk) as 'Ügyfél leghosszabb foglalása' 
	from foglalas JOIN vendeg ON foglalas.ugyfel_fk = vendeg.usernev
	ORDER BY vendeg.nev, foglalas.mettol;''' , engine)
print(df_query)

                 nev  FOGLALAS_PK  Foglalás hossza  \
0       Ambrus Dávid          653                7   
1       Ambrus Dávid          938                1   
2       Ambrus Dávid          994                7   
3       Ambrus Dávid          681                3   
4       Ambrus Dávid          757                7   
...              ...          ...              ...   
1022  Zatykó Katalin          665                7   
1023  Zatykó Katalin          723                7   
1024  Zatykó Katalin          751                4   
1025  Zatykó Katalin         1192                2   
1026  Zatykó Katalin         1221                2   

      Ügyfél legrövidebb foglalása  Ügyfél leghosszabb foglalása  
0                                1                             7  
1                                1                             7  
2                                1                             7  
3                                1                             7  
4               

In [8]:
# 03. Készítsük listát, amely megjeleníti az ügyfelek azonosítóját, nevét és a foglalásainak kezdő- és befejező dátumát, 
# valamint azt, hogy a foglalás összesen hány főre történt! 
# a. Egy új oszlopban jelenítsük meg azt is, hogy az ügyfél előző foglalása hány főre történt!

df_query = pd.read_sql_query('''select f.ugyfel_fk, v.nev, f.mettol, f.meddig,
	(f.felnott_szam + f.gyermek_szam) as 'FoglalásHányFőre' , 	
	lag(f.felnott_szam + f.gyermek_szam,1,0) over(partition by f.ugyfel_fk order by f.foglalas_pk) 'Előző Foglalás Hány főre'
	from vendeg v join foglalas f on v.usernev = f.ugyfel_fk
	order by 'FoglalásHányFőre';''' , engine)
print(df_query)

      ugyfel_fk                nev      mettol      meddig  FoglalásHányFőre  \
0         adam1          Kiss Ádám  2016-08-20  2016-08-24                 1   
1         adam3       Barkóci Ádám  2017-02-06  2017-02-10                 1   
2         agnes      Lengyel Ágnes  2016-05-25  2016-05-30                 1   
3        agnes3  Hartyánszky Ágnes  2016-08-04  2016-08-10                 1   
4        AGNESH      Horváth Ágnes  2016-06-23  2016-06-26                 1   
...         ...                ...         ...         ...               ...   
1022  sebestyen      Rab Sebestyén  2016-10-26  2016-11-01                 5   
1023   szilvia1       Tari Szilvia  2016-12-25  2016-12-29                 5   
1024    tihamer       Kazy Tihamér  2016-06-03  2016-06-08                 5   
1025      tunde      Turcsik Tünde  2017-01-03  2017-01-06                 5   
1026    zoltan4       Barna Zoltán  2016-10-21  2016-10-24                 5   

      Előző Foglalás Hány főre  
0     

In [9]:
# 04. Készítsünk listát arról, hogy hányszor foglaltak szállást évente a különböző csillaggal minősített szállásokon?
# Jelenjenek meg: a csillagok_száma, a szálláshely neve, az év, és "Foglalások száma"
# A lista ne tartalmazzon duplikált sorokat!

df_query = pd.read_sql_query('''select distinct Szallashely.szallas_nev ,szallashely.csillagok_Szama, YEAR(Foglalas.METTOL) Év, 
count(*) over(partition by Szallashely.szallas_nev, year(Foglalas.mettol), szallashely.csillagok_Szama )'Foglalások száma' 
from Szallashely join szoba on Szallashely.szallas_id = Szoba.szallas_fk 
						join Foglalas on szoba.szoba_id = foglalas.SZOBA_FK
						join vendeg on foglalas.ugyfel_fk = vendeg.usernev
						Order by szallashely.csillagok_Szama, Év;''' , engine)
print(df_query)

             szallas_nev  csillagok_Szama    Év  Foglalások száma
0            Családi Ház                0  2016                13
1       Fortuna Apartman                0  2016                35
2    Partiszél Vendégház                0  2016                23
3               Sába-Ház                0  2016                63
4   Szieszta Apartmanház                0  2016                28
5            Családi Ház                0  2017                 5
6       Fortuna Apartman                0  2017                 4
7    Partiszél Vendégház                0  2017                 3
8               Sába-Ház                0  2017                 7
9   Szieszta Apartmanház                0  2017                 6
10        Fortuna panzió                3  2016               104
11            Gold Hotel                3  2016               152
12           Jáde panzió                3  2016                24
13         Kentaur Hotel                3  2016               150
14        

In [10]:
#05. Oldjuk mega feladatot úgy, hogy Év szerint jelenjenek meg a részösszegek, és a végösszeg is
df_query = pd.read_sql_query('''SELECT GROUPING_ID(szallashely.szallas_nev, szallashely.csillagok_szama,yEAR(foglalas.mettol)) AS 'groupID',
		 (case grouping_id(szallashely.szallas_nev, szallashely.csillagok_szama,yEAR(foglalas.mettol)) 
					when 0 then  szallashely.szallas_nev
					when 4 then 'Szálláshelyek * szerint ' 
					when 6 then 'Szálláshelyek év szerint ' 
					when 7 then ' ** VÉGÖSSZEG ** ' 
					ELSE CAST(szallashely.szallas_nev AS NVARCHAR(15))
					END 
			) as 'Szálláshely neve',
		(case grouping_id(szallashely.szallas_nev, szallashely.csillagok_szama,yEAR(foglalas.mettol)) 
					when 0 then CAST(szallashely.csillagok_szama AS NVARCHAR(1)) 
					when 4 then CAST(szallashely.csillagok_szama AS NVARCHAR(1))  + ' csillagos '
					when 6 then CAST(YEAR(foglalas.mettol) AS NVARCHAR) + ' összes' 
					when 7 then ' ' 
					ELSE CAST(szallashely.csillagok_szama AS NVARCHAR(1))
					END ) as 'Csillagok száma',
		COUNT(*) AS 'Foglalások száma' 
	FROM Szallashely 
	JOIN Szoba ON Szallashely.szallas_id = Szoba.szallas_fk 
	JOIN Foglalas ON Szoba.szoba_id = Foglalas.SZOBA_FK
	GROUP BY GROUPING SETS (
		(szallashely.csillagok_szama, szallashely.szallas_nev, YEAR(foglalas.mettol)),
		(szallashely.csillagok_szama, YEAR(foglalas.mettol)), 
		(YEAR(foglalas.mettol)), 
		())
		ORDER BY 'Foglalások száma', /*Év, */ szallashely.csillagok_szama;''' , engine)
print(df_query)

    groupID           Szálláshely neve Csillagok száma  Foglalások száma
0         0        Partiszél Vendégház               0                 3
1         0           Fortuna Apartman               0                 4
2         0                Családi Ház               0                 5
3         0                Jáde panzió               3                 5
4         0       Szieszta Apartmanház               0                 6
5         0                   Sába-Ház               0                 7
6         0                Családi Ház               0                13
7         0               Lagúna Hotel               4                20
8         0             Fortuna panzió               3                22
9         0        Partiszél Vendégház               0                23
10        0                Jáde panzió               3                24
11        4   Szálláshelyek * szerint     0 csillagos                 25
12        0                 Gold Hotel             

In [11]:
# 06.Készítsünk lekérdezést, amely megmutatja, hogy hány db szálláshely tartozik 
# az 0, 3, 4  csillagos szálláshelyek közé?
# A lista a csillaggal jelölt kategória nevét és a darabszámot jelenítse meg, valamint ne tartalmazzon duplikált sorokat. 

df_query = pd.read_sql_query('''select distinct szallashely.csillagok_Szama, 
	count(SZALLAS_ID) over(partition by szallashely.csillagok_Szama order by szallashely.csillagok_Szama) as 'Szálláshelyek száma'
	from Szallashely join szoba on Szallashely.szallas_id = Szoba.szallas_fk 
						join Foglalas on szoba.szoba_id = foglalas.SZOBA_FK
						join vendeg on foglalas.ugyfel_fk = vendeg.usernev
						order by 'Szálláshelyek száma';''' , engine)
print(df_query)

   csillagok_Szama  Szálláshelyek száma
0                0                  187
1                4                  327
2                3                  513


In [12]:
# 07 Listázzuk ki azon vendégek nevét akik a legkevesebb ideig maradtak az egyes szálláshelyeken!
# A lekérdezésben szerepeljen a szállás neve, típusa, Tartózkodás ideje.

df_query = pd.read_sql_query('''select szallashely.tipus, Szallashely.SZALLAS_NEV, vendeg.nev,  DATEDIFF(DAY, foglalas.mettol, foglalas.meddig) AS minTartózkodás
		from Szallashely join szoba on Szallashely.szallas_id = Szoba.szallas_fk 
						join Foglalas on szoba.szoba_id = foglalas.SZOBA_FK
						join vendeg on foglalas.ugyfel_fk = vendeg.usernev
						where DATEDIFF(DAY, foglalas.mettol, foglalas.meddig) = 
																				(select min(DATEDIFF(DAY, foglalas.mettol, foglalas.meddig)) 
														from Szallashely join szoba on Szallashely.szallas_id = Szoba.szallas_fk 
																				join Foglalas on szoba.szoba_id = foglalas.SZOBA_FK
																				join vendeg on foglalas.ugyfel_fk = vendeg.usernev /**/) 
																				 order by szallashely.tipus, minTartózkodás;''' , engine)
print(df_query)

         tipus       SZALLAS_NEV                 nev  minTartózkodás
0     Apartman  Fortuna Apartman    Nagymihály Lajos               1
1     Apartman  Fortuna Apartman         Kiss Andrea               1
2     Apartman  Fortuna Apartman         Móra László               1
3     Apartman  Fortuna Apartman        Szűcs Balázs               1
4     Apartman  Fortuna Apartman     Harangozó János               1
..         ...               ...                 ...             ...
145  vendégház          Sába-Ház         Vígh András               1
146  vendégház          Sába-Ház  Foltényi Sebastián               1
147  vendégház       Családi Ház          Bagi Beáta               1
148  vendégház          Sába-Ház         Farkas Sára               1
149  vendégház       Családi Ház     Tatár Nikoletta               1

[150 rows x 4 columns]


In [13]:
# 08 Hányan foglaltak szállást (felnőtt + gyerek) 
# - havi bontásban az egyes szálláshelyeken, 
# külön csoportosítva:
# - a pótágyas 
# - a klímás 
# - a semmi
# - és a pótágyas+ klímás szobákra nézve?
# Részösszegek és végösszegeket is tünessünk fel.

df_query = pd.read_sql_query(''' SELECT grouping_id(szallashely.SZALLAS_NEV, MONTH(foglalas.mettol)) AS 'groupID',
    CASE 
		when grouping_id(szallashely.SZALLAS_NEV, MONTH(foglalas.mettol))= 1  then szallashely.szallas_nev
		when grouping_id(szallashely.SZALLAS_NEV, MONTH(foglalas.mettol))= 2  then CAST(MONTH(foglalas.mettol) AS NVARCHAR)
		when grouping_id(szallashely.SZALLAS_NEV, MONTH(foglalas.mettol))= 3  then 
		case 
			WHEN szoba.KLIMAS  = 'i' AND szoba.potagy > 0 THEN '* Pótágyas + Klímás'
			WHEN szoba.potagy > 0 and szoba.KLIMAS= 'n' THEN '* Pótágyas'
			WHEN szoba.potagy = 0 and szoba.KLIMAS = 'i' THEN '* Klímás'
			WHEN szoba.KLIMAS  = 'n' AND szoba.potagy = 0 THEN '* Semmi'
			Else '*** Végösszeg ***'
		end 
    END AS 'Szálláshely/Szoba típusa/Hónap',
    SUM(Foglalas.FELNOTT_SZAM + Foglalas.GYERMEK_SZAM) AS 'Szállásfoglalók száma'
FROM Szallashely 
JOIN Szoba ON Szallashely.szallas_id = Szoba.szallas_fk 
JOIN Foglalas ON Szoba.szoba_id = foglalas.SZOBA_FK 
JOIN Vendeg ON foglalas.ugyfel_fk = vendeg.usernev
GROUP BY GROUPING SETS (
    (szallashely.SZALLAS_NEV), 
    (MONTH(foglalas.mettol)), 
    (szoba.KLIMAS, szoba.potagy),
    ())  ORDER BY 'groupID';''' , engine)
print(df_query)

    groupID Szálláshely/Szoba típusa/Hónap  Szállásfoglalók száma
0         1           Szieszta Apartmanház                     83
1         1                 Fortuna Panzió                    311
2         1                  Kentaur Hotel                    420
3         1            Partiszél Vendégház                     63
4         1                       Sába-Ház                    166
5         1                    Családi Ház                     52
6         1               Fortuna Apartman                    106
7         1             Hotel Három Hattyú                    504
8         1                    Jáde panzió                     66
9         1                   Lagúna Hotel                    291
10        1                     Gold Hotel                    428
11        2                              9                    190
12        2                              3                     12
13        2                             12                    174
14        

In [14]:
# 09 Hány olyan foglalás történt, ahol csak úgy fértek el a vendégek, hogy pótágyat kellett volna használni (és volt is)?
# Tüntessük fel ezen szállások nevét!

df_query = pd.read_sql_query('''select szallashely.szallas_nev, count(foglalas.FOGLALAS_PK) AS 'Pótágyas foglalások száma'
from szallashely join szoba on szallas_id = szallas_fk join Foglalas on SZOBA_id = SZOBA_FK
where Szoba.FEROHELY < (foglalas.FELNOTT_SZAM+foglalas.gyermek_szam) and szoba.POTAGY > 0
group by szallashely.szallas_nev order by SZALLAS_NEV;''' , engine)
print(df_query)


            szallas_nev  Pótágyas foglalások száma
0           Családi Ház                          8
1      Fortuna Apartman                         13
2        Fortuna panzió                         24
3            Gold Hotel                         31
4    Hotel Három Hattyú                         54
5         Kentaur Hotel                         43
6          Lagúna Hotel                         30
7   Partiszél Vendégház                          5
8              Sába-Ház                         15
9  Szieszta Apartmanház                          3


In [15]:
# 10. Készítsünk csoportokat a foglalást leadott vendég száma alapján ( felnőtt+gyerek): 
#  - A 3 v. annál kevesebb foglalók számának értéke legyen: "kevés" 
#  - A 4-5 közötti "átlagos", 
#  - Az 5 feletti legyen a "sok"
# Listázzuk az egyes csoportokat, és a csoportokba tartozó vendégek számát végösszeggel együtt! 

df_query = pd.read_sql_query('''SELECT IIF(  
	GROUPING_id(CASE 
				WHEN (felnott_szam + gyermek_szam) <= 2 THEN 'kevés'
				WHEN (felnott_szam + gyermek_szam) BETWEEN 3 AND 4 THEN 'átlagos'
				ELSE 'sok' END)=1, 'Végösszeg', CAST(
													 (CASE 
														WHEN (felnott_szam + gyermek_szam) <= 2 THEN 'kevés'
														WHEN (felnott_szam + gyermek_szam) BETWEEN 3 AND 4 THEN 'átlagos'
														ELSE 'sok'
													END) AS nvarchar(10))  ) AS 'Vendég csoport',
    COUNT(*) AS 'Vendégek száma'
FROM foglalas
GROUP BY GROUPING SETS (
    (CASE 
        WHEN (felnott_szam + gyermek_szam) <= 2 THEN 'kevés'
        WHEN (felnott_szam + gyermek_szam) BETWEEN 3 AND 4 THEN 'átlagos'
        ELSE 'sok' END),())
	ORDER BY 'Vendégek száma';''' , engine)
print(df_query)

  Vendég csoport  Vendégek száma
0            sok              12
1        átlagos             454
2          kevés             561
3      Végösszeg            1027


In [16]:
# 11. Készítsünk listát az vendégek adatairól név szerinti sorrendben. 
#  Minden sorban jelenjen meg a sorrend szerint előző, illetve következő vendég neve is.
#  Ha nincs előző vagy következő vendég, akkor a 'Nincs' jelenjen meg!

df_query = pd.read_sql_query('''SELECT 
    usernev AS 'Vendég neve',
    ISNULL(LAG(usernev) OVER (ORDER BY usernev), 'Nincs') AS 'Előző vendég',
    ISNULL(LEAD(usernev) OVER (ORDER BY usernev), 'Nincs') AS 'Következő vendég'
FROM vendeg
ORDER BY usernev;''' , engine)
print(df_query)

    Vendég neve Előző vendég Következő vendég
0         adam1        Nincs            adam3
1         adam3        adam1            adam4
2         adam4        adam3            agnes
3         agnes        adam4           agnes3
4        agnes3        agnes           AGNESH
..          ...          ...              ...
191      ZSOLTJ       zsolt1           zsuzsa
192      zsuzsa       ZSOLTJ          zsuzsa3
193     zsuzsa3       zsuzsa        zsuzsanna
194   zsuzsanna      zsuzsa3          ZSUZSAV
195     ZSUZSAV    zsuzsanna            Nincs

[196 rows x 3 columns]


In [18]:
# 12.Készítsünk listát a vendégek számáról a foglalás éve, azon belül hónapja szerint csoportosítva.
# 	A lista tartalmazza a részösszegeket és a végösszegeket is.	Nevezzük el az oszlopokat.

df_query = pd.read_sql_query('''SELECT 
			GROUPING_ID(YEAR(foglalas.mettol), MONTH(foglalas.mettol)) AS 'GroupingID',
		CASE 
			GROUPING_ID(YEAR(foglalas.mettol), MONTH(foglalas.mettol))
				WHEN 0 THEN CAST(MONTH(foglalas.mettol) AS nvarchar(4))
				WHEN 1 THEN CONCAT( YEAR(foglalas.mettol), ' évre összesen') 
				when 2 then DATENAME(MONTH, DATEADD(MONTH, month(foglalas.mettol) - 1, '2000-01-01')) + ' hónap foglalásai'
				WHEN 3 THEN 'Grand Total'
			END AS 'Hónap', 
			COUNT(*) AS 'Vendégszám' 
		FROM Foglalas 
		JOIN vendeg ON foglalas.ugyfel_fk = vendeg.usernev 
		GROUP BY CUBE(YEAR(foglalas.mettol), MONTH(foglalas.mettol)) 
		HAVING GROUPING_ID(YEAR(foglalas.mettol), MONTH(foglalas.mettol)) >= 1
		ORDER BY COUNT(*);''' , engine)
print(df_query)


    GroupingID                       Hónap  Vendégszám
0            2      March hónap foglalásai           5
1            2      April hónap foglalásai          53
2            2  September hónap foglalásai          76
3            2   December hónap foglalásai          76
4            2    January hónap foglalásai          77
5            2        May hónap foglalásai          88
6            2   February hónap foglalásai          93
7            2     August hónap foglalásai          99
8            2   November hónap foglalásai         100
9            2       June hónap foglalásai         110
10           2       July hónap foglalásai         113
11           2    October hónap foglalásai         137
12           1          2017 évre összesen         175
13           1          2016 évre összesen         852
14           3                 Grand Total        1027


In [17]:
# 13. Készítsünk listát 'AGNESH' nevű felhasználóNK összes foglalásának számáról éves, azon belül havi bontásban! 
# 			a. A lista megfelelően jelölve tartalmazza a rész- és összdarabszámot is!
# 			b. Az oszlopokat nevezzük el értelemszerűen! És rendezzük a foglalás ideje alapján csökkenő sorrendbe.

df_query = pd.read_sql_query('''select IIF(Grouping_ID(year(foglalas.mettol))=1,'Összdarabszám', cast(year(foglalas.mettol) as nvarchar(5))) 'Év', 
	IIF(Grouping_ID(month(foglalas.mettol))=1,'Havi részlet', cast(month(foglalas.mettol) as nvarchar(5))) 'Hó',
	count(foglalas.foglalas_pk) 'DB' 
	from vendeg join Foglalas on foglalas.ugyfel_fk = vendeg.usernev where ugyfel_fk = 'AGNESH' 
	group by rollup(year(foglalas.mettol), month(foglalas.mettol)) order by year(foglalas.mettol) Desc;''' , engine)
print(df_query)

              Év            Hó  DB
0           2017             1   2
1           2017  Havi részlet   2
2           2016             4   2
3           2016             6   1
4           2016            10   1
5           2016  Havi részlet   4
6  Összdarabszám  Havi részlet   6


In [22]:
# 14. listázzuk az összes vendég foglalásának számát év, majd azon belül hó szerint. Rendezzük DB szerint növekvő sorrendbe 

df_query = pd.read_sql_query('''select IIF(Grouping_ID(year(foglalas.mettol))=1,'végDBszám', cast(year(foglalas.mettol) as nvarchar(5))) 'Év',
		   IIF(Grouping_ID(month(foglalas.mettol))=1,'részDBszám', cast(month(foglalas.mettol) as nvarchar(5))) 'Hó',
	count(foglalas.foglalas_pk) DB
	from vendeg join Foglalas on foglalas.ugyfel_fk = vendeg.usernev  group by rollup(year(foglalas.mettol), month(foglalas.mettol)) order by DB;''' , engine)
print(df_query)

           Év          Hó    DB
0        2017           3     5
1        2016           4    53
2        2016           9    76
3        2016          12    76
4        2017           1    77
5        2016           5    88
6        2017           2    93
7        2016           8    99
8        2016          11   100
9        2016           6   110
10       2016           7   113
11       2016          10   137
12       2017  részDBszám   175
13       2016  részDBszám   852
14  végDBszám  részDBszám  1027


In [21]:
# 15. Szállástípusonként és csillagok száma szerint hány db foglalás történt + összfoglalások száma is! Foglalások száma szerint rendezzük növekvő sorrendbe 

df_query = pd.read_sql_query('''SELECT 
    CASE 
        WHEN GROUPING(szallashely.TIPUS) = 1 AND GROUPING(szallashely.csillagok_szama) = 0 THEN '* Részfoglalás *'
        WHEN GROUPING(szallashely.TIPUS) = 1 AND GROUPING(szallashely.csillagok_szama) = 1 THEN 'Végösszeg'
        ELSE szallashely.TIPUS 
    END AS 'Szállástípus',
    CASE 
        WHEN GROUPING(szallashely.csillagok_szama) = 1 THEN ' - '
        ELSE CAST(szallashely.csillagok_szama AS VARCHAR)
    END AS 'Csillagok száma',
    COUNT(foglalas.foglalas_pk) AS 'Foglalások száma'
FROM Szallashely JOIN szoba ON Szallashely.szallas_id = Szoba.szallas_fk JOIN Foglalas ON szoba.szoba_id = foglalas.SZOBA_FK 
GROUP BY GROUPING SETS ((szallashely.TIPUS, csillagok_szama),(szallashely.csillagok_szama),()) 
ORDER BY count(foglalas.foglalas_pk);''' , engine)
print(df_query)

       Szállástípus Csillagok száma  Foglalások száma
0          Apartman               0                73
1         vendégház               0               114
2            panzió               3               155
3  * Részfoglalás *               0               187
4             Hotel               4               327
5  * Részfoglalás *               4               327
6             Hotel               3               358
7  * Részfoglalás *               3               513
8         Végösszeg              -               1027


In [20]:
# 16. Osztályozzuk a szálláshelyeket 4 típusba és rendezzük őket férőhely szerint növekvő sorrende, de csak azokat ahol nincs klíma:

df_query = pd.read_sql_query('''select distinct szallas_nev, klimas, ferohely, tipus, NTILE(4) over(partition by tipus order by szoba.ferohely) 'Szálláshelyosztályok' 
	from szoba join szallashely on szallas_id = szallas_fk where klimas = 'n';''' , engine)
print(df_query)

             szallas_nev klimas  ferohely       tipus  Szálláshelyosztályok
0           Bagoly Hotel      n         1       Hotel                     1
1           Bagoly Hotel      n         2       Hotel                     1
2           Bagoly Hotel      n         3       Hotel                     4
3       Fortuna Apartman      n         2    Apartman                     1
4       Fortuna Apartman      n         3    Apartman                     2
5       Fortuna Apartman      n         3    Apartman                     3
6       Fortuna Apartman      n         3    Apartman                     4
7         Fortuna Panzió      n         2      panzió                     1
8         Fortuna panzió      n         2      panzió                     2
9         Fortuna Panzió      n         3      panzió                     3
10            Gold Hotel      n         2       Hotel                     1
11            Gold Hotel      n         2       Hotel                     2
12          

In [19]:
# 17. A szálláshelyenként és szállástípusonként számoljuk össze kik voltak azok a vendégek akik pótágyas szobában szálltak meg?

df_query = pd.read_sql_query('''select distinct szallashely.szallas_nev, szallashely.tipus ,
	count(vendeg.usernev) over (partition by szallashely.szallas_nev ) 'Vendégek száma szállásonként' ,
	count(vendeg.usernev) over (partition by szallashely.tipus ) 'Vendégek száma típusonként'
	from Szallashely join szoba on Szallashely.szallas_id = Szoba.szallas_fk 
	join Foglalas on szoba.szoba_id = foglalas.SZOBA_FK
	join vendeg on foglalas.ugyfel_fk = vendeg.usernev where szoba.potagy > 0 
	order by szallashely.tipus, szallashely.szallas_nev;''' , engine)
print(df_query)

            szallas_nev      tipus  Vendégek száma szállásonként  \
0      Fortuna Apartman   Apartman                            31   
1  Szieszta Apartmanház   Apartman                             9   
2            Gold Hotel      Hotel                            79   
3    Hotel Három Hattyú      Hotel                           119   
4         Kentaur Hotel      Hotel                            99   
5          Lagúna Hotel      Hotel                            72   
6        Fortuna panzió     panzió                            56   
7           Családi Ház  vendégház                            14   
8   Partiszél Vendégház  vendégház                            11   
9              Sába-Ház  vendégház                            37   

   Vendégek száma típusonként  
0                          40  
1                          40  
2                         369  
3                         369  
4                         369  
5                         369  
6                          